In [ ]:
from __future__ import annotations

import os, re
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup(p: Path) -> None:
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)

# ------------------------------------------------------------
# 1) Patch L0 node in src/careeragent/langgraph/nodes.py
#    Make it local-only, no tool calls, always returns immediately.
# ------------------------------------------------------------
nodes_path = ROOT / "src/careeragent/langgraph/nodes.py"
assert nodes_path.exists(), f"Missing {nodes_path}"
backup(nodes_path)

txt = nodes_path.read_text(encoding="utf-8")

new_l0 = r'''
async def l0_security_node(state: dict) -> dict:
    """
    Description: Fast, local-only security gate. No network calls, no heavy imports.
    Layer: L0
    Input: resume_text + preferences
    Output: live_feed + attempts + (blocked?) status updates
    """
    resume = (state.get("resume_text") or "").lower()

    signals = [
        "ignore previous instructions",
        "system prompt",
        "developer message",
        "jailbreak",
        "exfiltrate",
        "reveal hidden",
    ]
    blocked = any(s in resume for s in signals)

    attempt = {
        "layer_id": "L0",
        "agent": "SanitizeAgent",
        "tool": "local.prompt_injection_heuristic",
        "model": None,
        "status": "failed" if blocked else "ok",
        "confidence": 0.10 if blocked else 0.95,
        "error": "prompt_injection" if blocked else None,
    }

    msg = "Security passed." if not blocked else "Blocked: prompt injection detected."

    delta = {
        "live_feed": [{"layer": "L0", "agent": "SanitizeAgent", "message": msg}],
        "attempts": [attempt],
    }

    if blocked:
        delta["status"] = "blocked"
        delta["pending_action"] = "security_blocked"

    return delta
'''

pattern = r"async def l0_security_node\([^\)]*\):.*?(?=\nasync def |\n# |\Z)"
if not re.search(pattern, txt, flags=re.S):
    raise RuntimeError("Could not find l0_security_node() in nodes.py to replace. Open nodes.py and search for it.")

txt2 = re.sub(pattern, new_l0.strip() + "\n\n", txt, flags=re.S)
nodes_path.write_text(txt2, encoding="utf-8")
print("✅ Patched L0 node:", nodes_path)

# ------------------------------------------------------------
# 2) Patch RunManagerService to use hard timeouts (thread executor)
# ------------------------------------------------------------
rm_path = ROOT / "src/careeragent/api/run_manager_service.py"
assert rm_path.exists(), f"Missing {rm_path}"
backup(rm_path)

rm_path.write_text(r'''
from __future__ import annotations

import asyncio
import concurrent.futures as cf
import copy
import os
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import run_single_layer, approve_ranking_flow, approve_drafts_flow

try:
    from careeragent.config import artifacts_root  # type: ignore
except Exception:
    def artifacts_root() -> Path:
        return Path("src/careeragent/artifacts").resolve()


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


class RunManagerService:
    """
    Description: Run manager with indestructible background execution + hard timeouts.
    Layer: L8
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(
            run_id=run_id,
            status=str(state.get("status", "unknown")),
            state=state,
            updated_at_utc=utc_now(),
        )

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)

        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            d = float(thresholds["default"])
            thresholds.setdefault("parser", d)
            thresholds.setdefault("discovery", d)
            thresholds.setdefault("match", d)
            thresholds.setdefault("draft", d)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {
                "created_at_utc": utc_now(),
                "heartbeat_utc": utc_now(),
                "last_layer": None,
                "pid": os.getpid(),
                "plan_layers": ["L0","L2","L3","L4","L5"],
            },
            "steps": [],
            "live_feed": [{"layer": "L1", "agent": "API", "message": "Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def start_background(self, run_id: str) -> None:
        t = threading.Thread(target=self._bg_sync_safe, args=(run_id,), daemon=True)
        t.start()

    def _bg_sync_safe(self, run_id: str) -> None:
        try:
            self._bg_sync(run_id)
        except Exception as e:
            st = self.get_state(run_id) or {"run_id": run_id}
            st["status"] = "failed"
            st["pending_action"] = "worker_crash"
            st.setdefault("live_feed", []).append({"layer":"L0","agent":"CrashGuard","message":f"Worker crashed: {e}"})
            self.save_state(run_id=run_id, state=st)

    def _run_layer_hard_timeout(self, state: Dict[str, Any], layer: str, timeout_sec: int) -> Dict[str, Any]:
        """
        Description: Run layer in isolated thread with hard timeout (cannot hang the runner).
        Layer: L6
        """
        def _call() -> Dict[str, Any]:
            st_copy = copy.deepcopy(state)
            return asyncio.run(run_single_layer(st_copy, layer))

        with cf.ThreadPoolExecutor(max_workers=1) as ex:
            fut = ex.submit(_call)
            try:
                return fut.result(timeout=timeout_sec)
            except cf.TimeoutError:
                state["status"] = "needs_human_approval"
                state["pending_action"] = f"timeout_{layer.lower()}"
                state.setdefault("live_feed", []).append({"layer": layer, "agent": "TimeoutGuard", "message": f"{layer} timed out after {timeout_sec}s."})
                return state
            except Exception as e:
                state["status"] = "failed"
                state["pending_action"] = f"error_{layer.lower()}"
                state.setdefault("live_feed", []).append({"layer": layer, "agent": "CrashGuard", "message": f"{layer} crashed: {e}"})
                return state

    def _bg_sync(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        plan = [("L0", 15), ("L2", 60), ("L3", 120), ("L4", 180), ("L5", 60)]

        for layer, tmo in plan:
            if state.get("status") in ("blocked", "needs_human_approval", "failed", "completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer

            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            state = self._run_layer_hard_timeout(state, layer, tmo)

            state["steps"][-1]["status"] = state.get("status", "running")
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"Orchestrator","message":"Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        # run actions without blocking FastAPI loop
        async def _run_async(fn):
            return await asyncio.to_thread(fn)

        if action_type == "execute_layer":
            layer = str(payload.get("layer", "")).upper()
            def _call():
                return asyncio.run(run_single_layer(copy.deepcopy(state), layer))
            state = await _run_async(_call)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_ranking":
            def _call():
                return asyncio.run(approve_ranking_flow(copy.deepcopy(state)))
            state = await _run_async(_call)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_drafts":
            def _call():
                return asyncio.run(approve_drafts_flow(copy.deepcopy(state)))
            state = await _run_async(_call)
            self.save_state(run_id=run_id, state=state)
            return state

        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''', encoding="utf-8")

print("✅ Patched L0 node + RunManager hard-timeout runner.")
print("Restart backend (no --reload) and start a NEW run.")